In [45]:
import pandas as pd
from rake_nltk import Rake
import seaborn as sns
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

raw = pd.read_csv('professionalsKeywords.csv')
raw = raw[raw.columns[~raw.columns.str.contains('Unnamed:')]] #purge empty columns

#count = CountVectorizer()
#count_matrix = count.fit_transform(df['answered_questions'])

# generating the cosine similarity matrix
#cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [46]:
'''
User I/O (Olivia)

- should be able to enter a pro id chosen at random and print 10 recommended questions

- (will probably require work from Destiny's code)

'''
#choose randomly from list of valid professionals
def randomProfessional():
    global raw
    return raw.sample()

#This is where the math happens
def getRecommendations(professional_id):
    return ['q1', 'q2', 'q3']

def main():
    choice = input("Enter a professional's ID for 10 recommendations...or just press enter for a random one.")
    if choice is None or choice == '': #idk which one happens when you enter nothing
        choice = randomProfessional()
        print('the professional you got is ')
        print(choice['professionals_id'].iloc[0])
        print('with these keywords')
        print(choice['answered_questions'].iloc[0])
    try:
        recs = getRecommendations(choice)
        print('10 questions to recommend to this professional:')
        for q in recs:
            print(q)
            
    except:
        print('had an oopsie...')
        
main()
        

Enter a professional's ID for 10 recommendations...or just press enter for a random one.
the professional you got is 
2c87f3ab51f24eb59675c0be103bc947
with these keywords
best strategy personally consider mcat study question need mainly asking next year hospital college start studying pre junior healthcare health care doctor med  doctor speed process becoming another 4 years school want medicine would really like least 3 residency doctor  working getting process surgical residency surgery medicine doctor residency life residence  mcats best time begin taking early junior year heard sophomore get wait please help begin second semester others say first medicine feel hand tests  long need degree take surgeon iv planning surgeon medicine become surgeons e pediatric residency  pursue radiology need degree question need still degree specifically radiology field medical able college go looked plan schools dream school currently academics early graduate high nursing premed care major associate

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [60]:

df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df = df[['Title','Genre','Director','Actors','Plot']]


df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    actors = row['Actors']
    director = row['Director']
    genre = row['Genre']
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    for item in [plot,actors,director,genre]:
        
    
        # extracting the words by passing the text
        r.extract_keywords_from_text(item)

        # getting the dictionary whith key words as keys and their scores as values
        key_words_dict_scores = r.get_word_degrees()
    
        # assigning the key words to the new column for the corresponding movie
        row['Key_words'] += ' '.join(list(key_words_dict_scores.keys()))
        

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)
df.drop(columns = ['Actors'], inplace = True)
df.drop(columns = ['Director'], inplace = True)
df.drop(columns = ['Genre'], inplace = True)

In [64]:
df = df.set_index('Title')
df.head()

,Key_words
Title,
The Shawshank Redemption,two imprisoned men bond number finding solace ...
The Godfather,reluctant son clandestine empire organized cri...
The Godfather: Part II,career expands vito corleone early life son mi...
The Dark Knight,mysterious past joker emerges ability chaos me...
12 Angry Men,jury holdout attempts colleagues miscarriage j...


In [68]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Key_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

recommendations('The Godfather')

['The Godfather: Part II',
 'Baby Driver',
 'The Night of the Hunter',
 'Fargo',
 'Rope',
 'Goodfellas',
 'On the Waterfront',
 'Scarface',
 'Anatomy of a Murder',
 'A Clockwork Orange']